In [ ]:
!pip uninstall -y pylibcudagraph-cu12 rmm-cu12 && pip install  pandas numpy pillow torch torchvision transformers scikit-learn timeout-decorator peft accelerate datasets bitsandbytes bert-score rouge-score rapidfuzz sentence-transformers evaluate

Found existing installation: rmm-cu12 25.2.0
Uninstalling rmm-cu12-25.2.0:
  Successfully uninstalled rmm-cu12-25.2.0
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.2 MB/s eta 0:00:00
   ━━━━

In [ ]:
import os
import gc
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
import torch
from transformers import (
    Blip2Processor,
    Blip2ForConditionalGeneration,
    AutoProcessor,
    Trainer,
    TrainingArguments
)


2025-05-17 05:55:19.374679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747461319.793481      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747461319.910884      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:

from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model
from tqdm import tqdm
from timeout_decorator import timeout, TimeoutError
from accelerate import Accelerator
from transformers.data.data_collator import default_data_collator
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    f1_score
)

In [ ]:
from bert_score import BERTScorer, score
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import evaluate

train_df = pd.read_csv('/kaggle/input/datasetfinal/train.csv')
val_df = pd.read_csv('/kaggle/input/datasetfinal/val.csv')
test_df = pd.read_csv('/kaggle/input/datasetfinal/test.csv')
print(f"Loaded training dataset with {len(train_df)} entries")
print(f"Loaded validation dataset with {len(val_df)} entries")
print(f"Loaded testing dataset with {len(test_df)} entries")
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))

Loaded training dataset with 80001 entries
Loaded validation dataset with 20001 entries
Loaded testing dataset with 20001 entries


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BLIP-2 model and processor 
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16).to(device)
model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

This function predicts a single-word answer to a given question about an image. It first checks if the image exists and loads it. Then, it creates a prompt emphasizing a single-word answer, processes the input, and generates the prediction using the model. The output is decoded, cleaned to keep only the last word, and returned. It also handles exceptions and cleans up GPU memory after processing.

In [ ]:
@timeout(10)
def predict_answer(image_path, question):
    try:

        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            return ""
        
        image = Image.open(image_path).convert("RGB")

        prompt = f"Give a single word answer for the Question: {question}. If you dont give single word answer you will be penalized heavily."

        inputs = processor(images=image, text=prompt, return_tensors="pt").to(device, torch.float16)

        with torch.no_grad():

            output_ids = model.generate(**inputs, max_new_tokens=20)

        predicted_answer = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        predicted_answer = predicted_answer.split()[-1] if predicted_answer else ""


        del inputs, output_ids

        gc.collect()
        torch.cuda.empty_cache()

        return predicted_answer
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

In [ ]:
import re
import inflect
inflect_engine = inflect.engine()

def normalize_answer(ans):
    ans = ans.strip().lower()
    ans = re.sub(r'[^\w\s]', '', ans)  # remove punctuation
    if ans.isdigit():
        ans = inflect_engine.number_to_words(ans)
    words = ans.split()
    return words[0] if words else "" 



Here we are creating a directory to save the prediction results if it doesn’t already exist. Then, we check if a prediction CSV file exists. If it does and contains data, we find out where we left off and resume from that point to avoid repeating work. Next, we prepare lists to store predictions, true answers, and other info. We loop through each row in the test dataset, skipping those already processed. For each image and question, we try to predict the answer, handling any timeouts by recording skipped entries. We save both the predicted and true answers, normalizing them to keep things consistent. Every 1000 rows or at the end, we save all the results to the CSV file, either creating it or appending to it if it already exists. After saving, we clear our temporary lists to keep memory usage low before continuing with the rest.

In [ ]:
import os
os.makedirs("/kaggle/working/resultCSV", exist_ok=True)
pred_path = '/kaggle/working/resultCSV/predictions.csv'
start_idx = 0
if os.path.exists(pred_path):
    try:
        existing = pd.read_csv(pred_path)

        if not existing.empty:
            start_idx = len(existing)

            print(f"Resuming from index {start_idx}")
    except pd.errors.EmptyDataError:

        print(f"Prediction file {pred_path} is empty, starting from index 0")
else:
    print(f"No existing prediction file found, starting from index 0")


predictions, results, y_true, y_pred, all_preds = [], [], [], [], []

ground_truths = test_df["answer"].tolist()

skipped_entries = []


for idx, row in tqdm(test_df.reset_index(drop=True).iterrows(), total=len(test_df), desc="Processing"):

    if idx < start_idx:
        continue

    full_image_path = row["full_image_path"]
    question = row["question"]
    try:

        predicted = predict_answer(full_image_path, question)

    except TimeoutError:
        print(f"Timeout at row {idx}: {full_image_path}")
        skipped_entries.append({"row": idx, "full_image_path": full_image_path, "question": question})
        predicted = ""


    predictions.append(predicted)

    y_true.append(str(ground_truths[idx]).lower())

    y_pred.append(str(predicted).lower())
    all_preds.append(str(predicted).lower())

    results.append({
        "img_path": full_image_path,
        "question": question,
        "true_answer": normalize_answer(str(ground_truths[idx]).lower()),
        "predicted_answer": normalize_answer(str(predicted).lower())
    })

    if (idx + 1) % 1000 == 0 or (idx + 1) == len(test_df):
        chunk_df = pd.DataFrame(results)

        if not os.path.exists(pred_path):
            chunk_df.to_csv(pred_path, mode='w', index=False, header=True)
        else:
            chunk_df.to_csv(pred_path, mode='a', index=False, header=False)

        print(f"Checkpoint saved at index {idx + 1}")
        
        results = []
        y_true = []
        y_pred = []

No existing prediction file found, starting from index 0


Processing:   5%|▍         | 1000/20001 [11:31<3:40:46,  1.43it/s]

Checkpoint saved at index 1000


Processing:  10%|▉         | 2000/20001 [22:46<3:14:21,  1.54it/s]

Checkpoint saved at index 2000


...

Checkpoint saved at index 19000


Processing: 100%|█████████▉| 20000/20001 [3:43:02<00:00,  1.31it/s]

Checkpoint saved at index 20000


Processing: 100%|██████████| 20001/20001 [3:43:03<00:00,  1.49it/s]

Checkpoint saved at index 20001


In [ ]:
# Normalize case
preds = pd.read_csv('/kaggle/working/resultCSV/predictions.csv')

preds = preds.dropna()

all_preds_BLIP = preds['predicted_answer'].tolist()

all_actuals_BLIP = preds['true_answer'].tolist()

In [ ]:
# metrics
preds_l = [p.lower() for p in all_preds_BLIP if isinstance(p, str)]
refs_l = [r.lower() for r in all_actuals_BLIP if isinstance(r, str)]

y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(all_actuals_BLIP)

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}")
print(f"Exact-match Recall:    {rec:.3f}")
print(f"Exact-match F1:        {f1:.3f}\n")

Exact-match Accuracy: 0.249
Exact-match Precision: 1.000
Exact-match Recall:    0.249
Exact-match F1:        0.398



In [11]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []

for pred, ref in zip(preds_l, refs_l):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

print(f"ROUGE Scores:")
print(f"ROUGE-1 F1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-L F1: {np.mean(rougeL_scores):.4f}")

ROUGE Scores:
ROUGE-1 F1: 0.2587
ROUGE-L F1: 0.2587


In [ ]:
# Sentence Embedding Cosine Similarity (memory-efficient pairwise)
model = SentenceTransformer('all-MiniLM-L6-v2')
pred_embeds = model.encode(preds_l, convert_to_tensor=True, device='cuda')
ref_embeds = model.encode(refs_l, convert_to_tensor=True, device='cuda')

cos_sims = [
    util.cos_sim(pred_embeds[i], ref_embeds[i]).item()
    for i in range(len(preds_l))
]

print(f"Sentence-BERT Cosine Similarity:")
print(f"Average Cosine Similarity: {np.mean(cos_sims):.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/617 [00:00<?, ?it/s]

Batches:   0%|          | 0/617 [00:00<?, ?it/s]

Sentence-BERT Cosine Similarity:
Average Cosine Similarity: 0.5522
